In [6]:
import numpy as np
import os
import open3d as o3d
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [32]:
# ディレクトリパス
directory_path = "/home/aichi2204/Documents/bkl2go/20240412-library2/aichi-20240412-library2_croped_mini_1/tmp"

# データを格納するリスト
data = []

# ディレクトリ内のすべてのpcdファイルについてループ
for filename in os.listdir(directory_path):
    if filename.endswith(".pcd"):
        print(filename)
        file_path = os.path.join(directory_path, filename)
        
        # PCDファイルの読み込み
        pcd = o3d.io.read_point_cloud(file_path)
        points = np.asarray(pcd.points)
        
        # 主成分分析の実行
        pca = PCA(n_components=3)
        pca.fit(points)
        
        # 固有値と固有ベクトルの取得
        eigenvalues = pca.explained_variance_
        eigenvectors = pca.components_.flatten()
        
        # 固有ベクトル方向の取りうる幅
        mean = np.mean(points, axis=0)
        centered_points = points - mean
        projected_data_1 = centered_points @ eigenvectors[6:9]
        range_1 = np.max(projected_data_1) - np.min(projected_data_1)
        print(range_1,projected_data_1.shape)
        
        # データをリストに追加
        data.append([filename[:-4], *eigenvalues, *eigenvectors])

# データフレームの作成
columns = ['id', 'r1', 'r2', 'r3'] + [f'pc{i}_{j}' for i in range(1, 4) for j in range(1, 4)]
df = pd.DataFrame(data, columns=columns)
df['id'] = df['id'].astype(int)
# CSVファイルに書き込み
output_csv_path = "output.csv"
df.to_csv(output_csv_path, index=False)

6.pcd
0.0336592130471486 (1716,)
4.pcd
0.0869076762803119 (27173,)
0.pcd
0.5998725097796987 (92514,)
11.pcd
0.04122907896748193 (3235,)
2.pcd
0.06931168667430343 (33307,)
1.pcd
1.5451746757239706 (75469,)
16.pcd
0.043250350097685096 (1862,)
10.pcd
0.04319024016537088 (10646,)
17.pcd
0.021591337335064676 (288,)
5.pcd
0.09175054521423193 (38992,)
7.pcd
0.04766192831429332 (2802,)
12.pcd
0.051776989890706476 (11455,)
15.pcd
0.04393005656237622 (2569,)
18.pcd
0.024186233942656492 (82,)
8.pcd
0.049997006887001705 (8949,)
14.pcd
0.038591162356602135 (1467,)
9.pcd
0.04772206469064455 (23889,)
13.pcd
0.04065944356661168 (7353,)
3.pcd
0.03047856319893119 (1726,)


In [12]:
# z軸のベクトル
x_axis = np.array([1, 0, 0])
y_axis = np.array([0, 1, 0])
z_axis = np.array([0, 0, 1])
# 角度を計算
def calculate_nas_angle(vector, axis):
    if axis == "x":
        angle = np.arccos(np.dot(vector, x_axis) / (np.linalg.norm(vector) * np.linalg.norm(x_axis)))
    if axis == "y":
        angle = np.arccos(np.dot(vector, y_axis) / (np.linalg.norm(vector) * np.linalg.norm(y_axis)))
    if axis == "z":
        angle = np.arccos(np.dot(vector, z_axis) / (np.linalg.norm(vector) * np.linalg.norm(z_axis)))
    return np.degrees(angle)

# 固有ベクトルを取得
id = 2
p1 = df.loc[df['id'] == id][['pc1_1', 'pc1_2', 'pc1_3']].values
p2 = df.loc[df['id'] == id][['pc2_1', 'pc2_2', 'pc2_3']].values
p3 = df.loc[df['id'] == id][['pc3_1', 'pc3_2', 'pc3_3']].values
print(df.loc[df['id'] == id][['r1', 'r2', 'r3']])
# print(calculate_nas_angle(p1,"x"))
# print(calculate_nas_angle(p1,"y"))
print(calculate_nas_angle(p1,"z"))
# print(calculate_nas_angle(p2,"x"))
# print(calculate_nas_angle(p2,"y"))
print(calculate_nas_angle(p2,"z"))
# print(calculate_nas_angle(p3,"x"))
# print(calculate_nas_angle(p3,"y"))
print(calculate_nas_angle(p3,"z"))

         r1        r2        r3
4  1.614296  0.007078  0.000008
[89.36794698]
[88.45225185]
[1.67188809]


In [22]:
df["sigma1"] = df["r1"] - df["r2"] 
df["sigma2"] = df["r2"] - df["r3"] 
df["sigma3"] = df["r3"]

In [24]:
df["d"] = df[['sigma1', 'sigma2', 'sigma3']].idxmax(axis=1).map({'sigma1': 1, 'sigma2': 2, 'sigma3': 3})

In [25]:
df.sort_values(by='id')

,id,r1,r2,r3,pc1_1,pc1_2,pc1_3,pc2_1,pc2_2,pc2_3,pc3_1,pc3_2,pc3_3,sigma1,sigma2,sigma3,d
2,0,0.492272,0.150995,0.007842,-0.677349,-0.593289,-0.434979,0.365623,0.241590,-0.898863,0.638372,-0.767882,0.053280,0.341277,0.143153,0.007842,1
5,1,1.380982,0.438482,0.120298,-0.341356,-0.847185,-0.407130,0.934955,-0.261514,-0.239729,-0.096625,0.462481,-0.881349,0.942500,0.318184,0.120298,1
4,2,1.614296,0.007078,0.000008,0.639300,-0.768878,0.011031,0.768454,0.639335,0.027010,-0.027820,-0.008791,0.999574,1.607218,0.007070,0.000008,1
18,3,0.050105,0.002975,0.000017,0.641567,-0.767046,-0.005654,0.007300,0.013476,-0.999883,-0.767032,-0.641450,-0.014245,0.047130,0.002958,0.000017,1
1,4,1.563128,0.006931,0.000008,-0.637645,0.770237,-0.011939,-0.769955,-0.637742,-0.021303,0.024022,0.004391,-0.999702,1.556198,0.006923,0.000008,1
9,5,1.600570,0.007172,0.000007,0.638381,-0.769642,0.010994,0.769309,0.638442,0.023574,-0.025162,-0.006591,0.999662,1.593398,0.007165,0.000007,1
0,6,0.070539,0.002697,0.000024,0.638112,-0.769941,-0.002112,-0.003403,-0.005563,0.999979,-0.769936,-0.638091,-0.006170,0.067842,0.002673,0.000024,1
10,7,0.102115,0.003081,0.000027,0.637290,-0.770576,-0.008627,-0.001539,0.009922,-0.999950,-0.770622,-0.637271,-0.005137,0.099034,0.003055,0.000027,1
14,8,0.310300,0.002793,0.000018,0.636754,-0.770968,0.012363,-0.009714,0.008011,0.999921,-0.771006,-0.636823,-0.002388,0.307507,0.002775,0.000018,1
16,9,1.275639,0.003143,0.000017,-0.637642,0.770231,-0.012527,-0.018454,0.000984,0.999829,-0.770112,-0.637764,-0.013586,1.272496,0.003126,0.000017,1


In [39]:
# 点群の読み込み
pcd = o3d.io.read_point_cloud("/home/aichi2204/Documents/bkl2go/20240412-library2/aichi-20240412-library2_croped_mini_1/tmp/0.pcd")
# 引数を設定して、平面を推定
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
# 平面モデルの係数を出力
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

# インライアの点を抽出して色を付ける
inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])

# 平面以外の点を抽出
outlier_cloud = pcd.select_by_index(inliers, invert=True)



Plane equation: -0.64x + 0.77y + -0.02z + -49.96 = 0


In [40]:
# 可視化
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])